In [1]:
#!pip install pandas duckdb duckdb-engine jupyterlab streamlit plotly
%pip install --upgrade duckdb duckdb-engine pandas pyarrow


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: /Users/dave/dev/RETrend/.env/bin/python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install --upgrade "pyspark==3.5.6"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.4/317.4 MB 21.5 MB/s eta 0:00:00m eta 0:00:010:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 25.3 MB/s eta 0:00:00
  DEPRECATION: pyspark is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for pyspark ... done

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: /Users/dave/dev/RETrend/.env/bin/python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
!ls -al ~/nfs_mount

total 17127
drwxrwxrwx   4 65534  65534     4096 Jun 25 12:53 .
-rw-r--r--   1 dave   staff     4096 Jun 25 12:54 ._complex_list.csv
-rw-r--r--   1 dave   staff     4096 Jun 25 12:53 ._eupmeandong_list.csv
-rw-r--r--   1 dave   staff     4096 Jun 24 23:22 ._iceberg
-rw-r--r--   1 dave   staff     4096 Jun 23 19:42 ._parquet
-rw-r--r--   1 dave   staff     4096 Jun 23 19:48 ._prquet
-rw-r--r--   1 dave   staff     4096 Jun 25 12:53 ._pyeonginfo_13337.csv
-rw-r--r--   1 dave   staff     4096 Jun 23 19:48 ._realestate.duckdb
-rw-r--r--   1 dave   staff     4096 Jun 25 12:53 ._shido_list.csv
-rw-r--r--   1 dave   staff     4096 Jun 25 12:53 ._shigungu_list.csv
-rw-r--r--   1 dave   staff     4096 Jun 23 19:25 ._test
drwxr-x---+ 41 dave   staff     1312 Jun 27 16:49 ..
-rw-r--r--@  1 dave   staff  8302844 Jun 21 23:51 complex_list.csv
-rw-r--r--@  1 dave   staff   337455 Jun 21 22:17 eupmeandong_list.csv
drwxr-xr-x@  3 dave   staff     4096 Jun 24 23:22 iceberg
drwxr-xr-x@  2 dave   staff  

In [23]:
# nfs server 개선작업으로 parquet 경로는 홈 폴더에 추가
# asis : ~/nfs_mount
# tobe : ~/dev/RETrend/tmp/data (/Users/dave/dev/RETrend/tmp/data)
raw_data_path="~/dev/RETrend/tmp/raw"

1. complex code 13337 에 대한 전체 기간 매매거래지표.csv 를 parquet으로 변환
2. 변환한 parquet을 TIMESTAMP_MILLIS 로 다시 쓴다.

In [28]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os

# 경로 설정
home          = os.path.expanduser("~")
raw_data_path = os.path.join(home, "dev/RETrend/tmp/raw")
csv_fp        = os.path.join(raw_data_path, "trade_history_13337.csv")
parquet_root  = os.path.join(raw_data_path, "parquet")
parquet_fp    = os.path.join(parquet_root, "trade_history.parquet")

# 디렉터리 보장
os.makedirs(parquet_root, exist_ok=True)

# 1) CSV → pandas.DataFrame
df = pd.read_csv(csv_fp)

# 2) date 컬럼 생성
df['date'] = pd.to_datetime(
    df['tradeYear'].astype(str) + '-' +
    df['tradeMonth'].astype(str).str.zfill(2) + '-' +
    df['tradeDate'].astype(str).str.zfill(2)
)

# 3) pandas → PyArrow Table
table = pa.Table.from_pandas(df)

# 4) Spark 호환 Parquet (밀리초 정밀도)
pq.write_table(
    table,
    parquet_fp,
    compression='snappy',
    coerce_timestamps='ms',   # logicalType = TIMESTAMP_MILLIS
    flavor='spark'            # Spark 호환
)

print("✅ Spark 호환 밀리초 Parquet 저장 완료:", parquet_fp)


✅ Spark 호환 밀리초 Parquet 저장 완료: /Users/dave/dev/RETrend/tmp/raw/parquet/trade_history.parquet


3. spark 으로 iceberg 테이블 생성 & 쓰기

In [30]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("IcebergWrite") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.3") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hadoop") \
    .config("spark.sql.catalog.spark_catalog.warehouse", "/Users/dave/dev/RETrend/tmp/raw/iceberg") \
    .config("spark.hadoop.fs.defaultFS", "file:///") \
    .getOrCreate()

# 1-1) parquet 로 변환해 둔 거래 이력 읽기
df = spark.read.parquet("file:///Users/dave/dev/RETrend/tmp/raw/parquet/trade_history_ms.parquet")

# 1-2) Iceberg 테이블 스키마 + 파티션(날짜) 생성 (최초 1회)
spark.sql("""
CREATE TABLE IF NOT EXISTS spark_catalog.default.trade_iceberg (
  complexNo                 BIGINT,
  tradeType                 STRING,
  tradeYear                 INT,
  tradeMonth                INT,
  tradeDate                 INT,
  dealPrice                 BIGINT,
  floor                     INT,
  representativeArea        DOUBLE,
  exclusiveArea             DOUBLE,
  formattedPrice            STRING,
  formattedTradeYearMonth   STRING,
  areaNo                    INT,
  deleteYn                  STRING,
  date                      DATE
)
USING iceberg
PARTITIONED BY (date)
""")

# 1-3) 데이터를 파티션 단위로 덮어쓰기
df.writeTo("spark_catalog.default.trade_iceberg") \
  .overwritePartitions()

spark.stop()
print("✅ Spark → Iceberg 쓰기 완료")


25/06/27 17:51:40 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
                                                                                

✅ Spark → Iceberg 쓰기 완료


4. DuckDB 에서 Iceberg 읽기

In [31]:
import duckdb

con = duckdb.connect('/Users/dave/realestate.duckdb')
con.execute("INSTALL iceberg; LOAD iceberg;")
con.execute("SET unsafe_enable_version_guessing = true;")

df = con.execute("""
  SELECT date, AVG(dealPrice) AS avg_price
  FROM iceberg_scan('/Users/dave/dev/RETrend/tmp/raw/iceberg/default/trade_iceberg')
  GROUP BY date
  ORDER BY date
""").df()

print(df)

          date  avg_price
0   2006-01-05    28000.0
1   2006-01-08    22900.0
2   2006-01-13    22700.0
3   2006-01-23    22275.0
4   2006-02-06    36000.0
..         ...        ...
825 2025-04-26    32000.0
826 2025-05-03    34000.0
827 2025-05-17    33300.0
828 2025-06-07    30200.0
829 2025-06-17    37000.0

[830 rows x 2 columns]


---

In [32]:
# 단지 위치 정보를 추가한다.

In [62]:
import duckdb, glob, os

# ─── 환경 설정 ─────────────────────────────────────────────────
home        = os.path.expanduser("~")
raw         = os.path.join(home, "dev/RETrend/tmp/raw")
parquet_dir = os.path.join(raw, "parquet")
os.makedirs(parquet_dir, exist_ok=True)

con = duckdb.connect()

# ─── (A) trade_history_{complexNo}.csv → Parquet ───────────────
for csv_fp in glob.glob(os.path.join(raw, "trade_history_*.csv")):
    fn        = os.path.basename(csv_fp)
    complexNo = int(fn.split("_")[2].split(".")[0])
    pq_fp     = os.path.join(parquet_dir, f"trade_history_{complexNo}.parquet")

    con.execute(f"""
    COPY (
      SELECT
        tradeType::VARCHAR                         AS tradeType,
        CAST(tradeYear       AS INTEGER)           AS tradeYear,
        CAST(tradeMonth      AS INTEGER)           AS tradeMonth,
        CAST(tradeDate       AS INTEGER)           AS tradeDate,
        CAST(dealPrice       AS INTEGER)           AS dealPrice,
        CAST(floor           AS INTEGER)           AS floor,
        CAST(representativeArea AS DOUBLE)         AS representativeArea,
        CAST(exclusiveArea   AS DOUBLE)            AS exclusiveArea,
        formattedPrice::VARCHAR                     AS formattedPrice,
        formattedTradeYearMonth::VARCHAR            AS formattedTradeYearMonth,
        CAST(areaNo         AS INTEGER)            AS areaNo,
        deleteYn::VARCHAR                           AS deleteYn,
        {complexNo}                                 AS complexNo,
        -- 날짜 컬럼: "YYYY-MM-DD 00:00:00" TIMESTAMP
        CAST(
          lpad(tradeYear::VARCHAR,4,'0') || '-' ||
          lpad(tradeMonth::VARCHAR,2,'0') || '-' ||
          lpad(tradeDate::VARCHAR,2,'0')
          AS TIMESTAMP
        ) AS date
      FROM read_csv_auto('{csv_fp}')
    ) 
    TO '{pq_fp}' (FORMAT PARQUET, COMPRESSION 'snappy');
    """)
    print("✅ CSV→Parquet:", fn)

# ─── (B) complex_list.csv → Parquet ────────────────────────────
complex_csv = os.path.join(raw, "complex_list.csv")
complex_pq  = os.path.join(parquet_dir, "complex_list.parquet")

con.execute(f"""
COPY (
  SELECT
    CAST(complexNo              AS INTEGER)  AS complexNo,
    complexName::VARCHAR                       AS complexName,
    CAST(cortarNo               AS BIGINT)    AS cortarNo,
    realEstateTypeCode::VARCHAR                AS realEstateTypeCode,
    realEstateTypeName::VARCHAR                AS realEstateTypeName,
    detailAddress::VARCHAR                     AS detailAddress,
    CAST(latitude              AS DOUBLE)     AS latitude,
    CAST(longitude             AS DOUBLE)     AS longitude,
    CAST(totalHouseholdCount   AS INTEGER)    AS totalHouseholdCount,
    CAST(totalBuildingCount    AS INTEGER)    AS totalBuildingCount,
    CAST(highFloor             AS INTEGER)    AS highFloor,
    CAST(lowFloor              AS INTEGER)    AS lowFloor,
    useApproveYmd::VARCHAR                     AS useApproveYmd,
    CAST(dealCount             AS INTEGER)    AS dealCount,
    CAST(leaseCount            AS INTEGER)    AS leaseCount,
    CAST(rentCount             AS INTEGER)    AS rentCount,
    CAST(shortTermRentCount    AS INTEGER)    AS shortTermRentCount,
    CAST(isInterest            AS BOOLEAN)    AS isInterest,
    cortarAddress::VARCHAR                      AS cortarAddress,
    CAST(tourExist             AS BOOLEAN)    AS tourExist,
    CAST(eupmeandongCortarNo   AS BIGINT)     AS eupmeandongCortarNo,
    eupmeandongCortarName::VARCHAR              AS eupmeandongCortarName
  FROM read_csv_auto('{complex_csv}')
)
TO '{complex_pq}' (FORMAT PARQUET, COMPRESSION 'snappy');
""")
print("✅ CSV→Parquet: complex_list.csv →", complex_pq)

con.close()

BinderException: Binder Error: Column "dealAmount" referenced that exists in the SELECT clause - but this column cannot be referenced before it is defined

In [61]:
# ─────────────────────────────────────────────────────────────────────────────
# Cell 1: 전체 필드를 명시한 Spark 스키마 정의 & Iceberg 테이블 생성
# ─────────────────────────────────────────────────────────────────────────────

import os
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType, StructField,
    LongType, IntegerType, StringType,
    DoubleType, TimestampType, BooleanType
)

# 1) 경로 설정
raw_data_path = os.path.expanduser("~/dev/RETrend/tmp/raw")
parquet_dir   = os.path.join(raw_data_path, "parquet")
iceberg_root  = os.path.join(raw_data_path, "iceberg")

# 2) SparkSession (Iceberg 설정 포함)
spark = (
    SparkSession.builder
        .appName("IcebergETL_FullSchema")
        .master("local[*]")
        .config("spark.driver.memory", "8g")
        .config("spark.executor.memory", "8g")
        .config("spark.sql.shuffle.partitions", "4")
        .config("spark.sql.extensions",
                "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .config("spark.sql.catalog.spark_catalog",
                "org.apache.iceberg.spark.SparkCatalog")
        .config("spark.sql.catalog.spark_catalog.type", "hadoop")
        .config("spark.sql.catalog.spark_catalog.warehouse",
                f"file://{iceberg_root}")
        .getOrCreate()
)

# 3) trade_history_full 스키마 (Parquet 에 저장된 타입에 맞춰 LongType/IntegerType etc. 으로 전부 명시)
trade_schema = StructType([
    StructField("tradeType",                StringType(),   True),
    StructField("dealPrice",                LongType(),     True),
    StructField("deposit",                  LongType(),     True),
    StructField("rent",                     LongType(),     True),
    StructField("tradeYear",                LongType(),     True),
    StructField("tradeMonth",               LongType(),     True),
    StructField("tradeDate",                LongType(),     True),
    StructField("formattedTradeYearMonth",  StringType(),   True),
    StructField("floor",                    LongType(),     True),
    StructField("areaNo",                   LongType(),     True),
    StructField("complexNo",                LongType(),  True),
    StructField("date",                     TimestampType(),True)
    # ─── 나머지 컬럼이 있다면 여기에도 모두 StructField("컬럼명", <Type>(), True) 로 추가 ───
])

# 4) complex_list 스키마 (full)
complex_schema = StructType([
    StructField("complexNo",               LongType(), True),
    StructField("complexName",             StringType(),  True),
    StructField("cortarNo",                LongType(),    True),
    StructField("realEstateTypeCode",      StringType(),  True),
    StructField("realEstateTypeName",      StringType(),  True),
    StructField("detailAddress",           StringType(),  True),
    StructField("latitude",                DoubleType(),  True),
    StructField("longitude",               DoubleType(),  True),
    StructField("totalHouseholdCount",     LongType(),    True),
    StructField("totalBuildingCount",      LongType(),    True),
    StructField("highFloor",               LongType(),    True),
    StructField("lowFloor",                LongType(),    True),
    StructField("useApproveYmd",           StringType(),  True),
    StructField("dealCount",               LongType(),    True),
    StructField("leaseCount",              LongType(),    True),
    StructField("rentCount",               LongType(),    True),
    StructField("shortTermRentCount",      LongType(),    True),
    StructField("isInterest",              BooleanType(),True),
    StructField("eupmeandongCortarNo",     LongType(),    True),
    StructField("eupmeandongCortarName",   StringType(),  True)
])

# 5) Parquet → DataFrame (스키마 강제)
trade_df = (
    spark.read
         .schema(trade_schema)
         .parquet(f"file://{parquet_dir}/trade_history_*.parquet")
)
complex_df = (
    spark.read
         .schema(complex_schema)
         .parquet(f"file://{parquet_dir}/complex_list.parquet")
)

# 6) Iceberg 테이블로 쓰기 (overwrite)
spark.sql("DROP TABLE IF EXISTS spark_catalog.default.trade_history")
trade_df.writeTo("spark_catalog.default.trade_history") \
        .using("iceberg") \
        .createOrReplace()

spark.sql("DROP TABLE IF EXISTS spark_catalog.default.complex_info")
complex_df.writeTo("spark_catalog.default.complex_info") \
          .using("iceberg") \
          .createOrReplace()

print("✅ Iceberg 테이블 생성 완료")


25/06/27 19:22:52 ERROR Utils: Aborting task
org.apache.spark.SparkException: Parquet column cannot be converted in file file:///Users/dave/dev/RETrend/tmp/raw/parquet/trade_history_13337.parquet. Column: [complexNo], Expected: bigint, Found: INT32.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:855)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:290)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:131)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:593)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBatch_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(Buf

Py4JJavaError: An error occurred while calling o1024.createOrReplace.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 26.0 failed 1 times, most recent failure: Lost task 0.0 in stage 26.0 (TID 43) (192.0.0.2 executor driver): org.apache.spark.SparkException: Parquet column cannot be converted in file file:///Users/dave/dev/RETrend/tmp/raw/parquet/trade_history_13337.parquet. Column: [complexNo], Expected: bigint, Found: INT32.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:855)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:290)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:131)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:593)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_